In [3]:
# import libraries
from bs4 import BeautifulSoup
import numpy as np
from time import sleep
from random import randint
from selenium import webdriver
import requests
import pandas as pd
from sentida import Sentida
import re


In [4]:

a_file = open("edc.txt", "r")
data = []
for line in a_file:
  stripped_line = line.strip()
  line_list = stripped_line.split()
  data.append(line_list)

a_file.close()

In [5]:
len(data)

2533

In [6]:
def flatten(t):
    return [item for sublist in t for item in sublist]

data = flatten(data)

In [15]:
# til  
description = [] 
price = []
Sgmprice = []
area = []
casenr = []
year = []


In [8]:
# Range er sat til 10 links ændre det for, at indlæse flere EDC links
for page in range(0,10):

    website = data[page]
    result = requests.get(website)
    content = result.text
    soup = BeautifulSoup(content, 'lxml')

    dec = soup.find('div', class_='description').text.strip()
    pr = soup.find('h2', class_='price').text.strip()
    sgm = soup.find(attrs={'class': 'value', 'data-qa': 'Pris/m²'}).text.strip()
    ar = soup.find(attrs={'class': 'value', 'data-qa': 'Boligareal'}).text.strip()
    case = soup.find(attrs={'class': 'value', 'data-qa': 'Sagsnr.'}).text.strip()
    yy = soup.find("p", {"data-qa":"Byggeår"}).get_text()

    
    description.append(dec)
    price.append(pr)
    Sgmprice.append(sgm)
    area.append(ar)
    casenr.append(case)
    year.append(yy)

In [9]:
df = pd.DataFrame(
    {'description': description,
     'price': price,
     'Sgmprice': Sgmprice,
     'area': area,
     'casenr': casenr,
     'year': year
    })


In [10]:
df

,description,price,Sgmprice,area,casenr,year
0,"Ejendommen sælges til nedrivning, da ingen af ...",399.000 kr.,1.995 kr.,202 m²,94401805,1887
1,"Her har du nu mulighed for at købe et flot, mo...",1.925.000 kr.,13.750 kr.,142 m²,94403598,1946
2,Villaen på Bryggervangen 5 har en rolig og cen...,1.699.000 kr.,11.799 kr.,133 m²,94403190,1964
3,Velkommen til Bundgårdsvej 2 i Biersted. Her f...,898.000 kr.,5.613 kr.,157 m²,94403501,1965
4,"Man kan måske godt se, at der bor børn i huset...",845.000 kr.,7.752 kr.,101 m²,94403028,1974
5,Nu har du muligheden for at erhverve dig denne...,2.995.000 kr.,7.488 kr.,332 m²,94403123,1949
6,"OBS: Der er fjernvarme.I er de allerførste, de...",2.695.000 kr.,17.967 kr.,150 m²,94402906,2020
7,"Projekt - under opførselI er de allerførste, d...",2.495.000 kr.,18.481 kr.,135 m²,94403266,2021
8,Velkommen til Enghaven 64 i Biersted.Her får d...,1.495.000 kr.,10.310 kr.,133 m²,94403509,1973
9,"Huset her er ret ideelt til et par, som vi bo ...",1.250.000 kr.,10.417 kr.,110 m²,94402948,1927


In [11]:
 # Create a function to get the polarity
def getPolarity(text):
    return Sentida().sentida(text, output="total")

In [12]:
df['score'] = df['description'].apply(getPolarity)

In [13]:
df

,description,price,Sgmprice,area,casenr,year,score
0,"Ejendommen sælges til nedrivning, da ingen af ...",399.000 kr.,1.995 kr.,202 m²,94401805,1887,1.738889
1,"Her har du nu mulighed for at købe et flot, mo...",1.925.000 kr.,13.750 kr.,142 m²,94403598,1946,18.169074
2,Villaen på Bryggervangen 5 har en rolig og cen...,1.699.000 kr.,11.799 kr.,133 m²,94403190,1964,12.830030
3,Velkommen til Bundgårdsvej 2 i Biersted. Her f...,898.000 kr.,5.613 kr.,157 m²,94403501,1965,13.497222
4,"Man kan måske godt se, at der bor børn i huset...",845.000 kr.,7.752 kr.,101 m²,94403028,1974,15.564074
5,Nu har du muligheden for at erhverve dig denne...,2.995.000 kr.,7.488 kr.,332 m²,94403123,1949,20.432465
6,"OBS: Der er fjernvarme.I er de allerførste, de...",2.695.000 kr.,17.967 kr.,150 m²,94402906,2020,9.725864
7,"Projekt - under opførselI er de allerførste, d...",2.495.000 kr.,18.481 kr.,135 m²,94403266,2021,9.313290
8,Velkommen til Enghaven 64 i Biersted.Her får d...,1.495.000 kr.,10.310 kr.,133 m²,94403509,1973,7.677407
9,"Huset her er ret ideelt til et par, som vi bo ...",1.250.000 kr.,10.417 kr.,110 m²,94402948,1927,17.794168


In [20]:
def cleanText(text):

    text = re.sub(r'kr.', '', text) #Remove the ( kr.) character
    text = re.sub(r'm²', '', text) #Remove the 'm²' character
    return text

df['price'] = df['price'].apply(cleanText)
df['Sgmprice'] = df['Sgmprice'].apply(cleanText)
df['area'] = df['area'].apply(cleanText)

In [21]:
df

,description,price,Sgmprice,area,casenr,year,score
0,"Ejendommen sælges til nedrivning, da ingen af ...",399.000,1.995,202,94401805,1887,1.738889
1,"Her har du nu mulighed for at købe et flot, mo...",1.925.000,13.750,142,94403598,1946,18.169074
2,Villaen på Bryggervangen 5 har en rolig og cen...,1.699.000,11.799,133,94403190,1964,12.830030
3,Velkommen til Bundgårdsvej 2 i Biersted. Her f...,898.000,5.613,157,94403501,1965,13.497222
4,"Man kan måske godt se, at der bor børn i huset...",845.000,7.752,101,94403028,1974,15.564074
5,Nu har du muligheden for at erhverve dig denne...,2.995.000,7.488,332,94403123,1949,20.432465
6,"OBS: Der er fjernvarme.I er de allerførste, de...",2.695.000,17.967,150,94402906,2020,9.725864
7,"Projekt - under opførselI er de allerførste, d...",2.495.000,18.481,135,94403266,2021,9.313290
8,Velkommen til Enghaven 64 i Biersted.Her får d...,1.495.000,10.310,133,94403509,1973,7.677407
9,"Huset her er ret ideelt til et par, som vi bo ...",1.250.000,10.417,110,94402948,1927,17.794168
